## Feature engineering

**Problem statement**: Given a user and a product (e.g. user 123 and product ID 98765) that a user has ordered on Instacart before, predict whether or not that user will reorder that product in their next Instacart order.

In [2]:
reset -fs

In [3]:
from collections import Counter
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('data/instacart.csv')

In [7]:
df.shape

(33819106, 15)

In [5]:
df.columns

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered',
       'product_name', 'aisle_id', 'department_id', 'aisle', 'department',
       'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order'],
      dtype='object')

### Indicate which `department`s contain foods that are considered staples or everyday items

In [75]:
df.department.unique()

array(['dairy eggs', 'produce', 'pantry', 'meat seafood', 'bakery',
       'personal care', 'snacks', 'breakfast', 'beverages', 'deli',
       'household', 'international', 'dry goods pasta', 'frozen',
       'canned goods', 'babies', 'pets', 'alcohol', 'bulk', 'missing',
       'other'], dtype=object)

In [80]:
staples = {
    'dairy eggs' : 1,
    'produce' : 1,
    'pantry' : 0,
    'meat seafood' : 1,
    'bakery' : 0,
    'personal care' : 0,
    'snacks' : 0,
    'breakfast' : 0,
    'beverages' : 0,
    'deli' : 0,
    'household' : 0,
    'international' : 0,
    'dry goods pasta' : 1,
    'frozen' : 0,
    'canned goods' : 0,
    'babies' : 0,
    'pets' : 0,
    'alcohol' : 0,
    'bulk' : 0,
    'missing' : 0,
    'other' : 0
}

In [92]:
df.loc[:, 'department_is_staple'] = df['department'].apply(lambda x: staples[x])

### Does the word 'organic' affect reordering rates

In [141]:
df_test = df.head(50000)

In [144]:
df.loc[:, 'product_is_organic'] = 0
df.loc[df.product_name.apply(lambda x: 'organic' in x.lower()), 'product_is_organic'] = 1

In [145]:
df.sample(10)

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,department_is_staple,product_is_organic
11141257,1176165,45603,5,0,Trilogy Kombucha Drink,31,7,refrigerated,beverages,23023,prior,2,0,23,3.0,0,0
14508583,1530890,28398,1,0,Master Brew Kombucha Ginger,31,7,refrigerated,beverages,161833,prior,5,1,16,26.0,0,0
28075259,2960600,22149,17,0,50% Reduced Fat Sharp Cheddar Cheese,21,16,packaged cheese,dairy eggs,195086,prior,4,6,12,19.0,1,0
8520785,899512,37067,3,1,Organic Banana,24,4,fresh fruits,produce,144206,prior,7,2,15,18.0,1,1
22855328,2410698,4605,1,1,Yellow Onions,83,4,fresh vegetables,produce,69866,prior,10,2,14,27.0,1,0
9285494,980072,33674,15,1,Elmo Mac 'n Cheese With Carrots & Broccoli,38,1,frozen meals,frozen,60810,prior,12,5,7,4.0,0,0
24608962,2595395,3298,1,1,Mozzarella String Cheese,21,16,packaged cheese,dairy eggs,53999,prior,26,4,9,10.0,1,0
29467974,3107879,45254,7,1,Organic Gala Apple,24,4,fresh fruits,produce,193460,prior,23,1,6,8.0,1,1
1060832,111985,38383,7,1,Organic White Onions,83,4,fresh vegetables,produce,73835,prior,15,5,12,1.0,1,1
7446047,786006,4276,18,0,Sea Salt Lentil Chips,107,19,chips pretzels,snacks,10670,prior,10,2,16,12.0,0,0


## Prototype -- only select 1% of rows from 'test' and 1% of rows from 'prior' to test out our feature engineering process

### Create truncated df to work with

In [104]:
df_trunc_prior = df_prior.sample(int(df_prior.shape[0] * 0.01), random_state=42)
df_trunc_train = df_train.sample(int(df_train.shape[0] * 0.01), random_state=42)

In [105]:
df_trunc = pd.concat([df_trunc_prior, df_trunc_train], axis=0)

In [106]:
df_trunc.shape

(338190, 15)

In [107]:
df_trunc.head(3)

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
29481110,3109255,34099,16,0,Crushed Red Chili Pepper,104,13,spices seasonings,pantry,135284,prior,9,0,19,8.0
2852353,301098,41950,5,0,Organic Tomato Cluster,83,4,fresh vegetables,produce,7293,prior,2,4,15,1.0
11194500,1181866,45066,8,0,Honeycrisp Apple,24,4,fresh fruits,produce,111385,prior,2,1,17,8.0


(122964, 314757)

In [119]:
# Top 10 number of orders by user
sorted(df_trunc.groupby(['user_id']).order_id.count(), reverse=True)[:10]

[44, 43, 37, 37, 36, 35, 33, 33, 33, 32]

### User-specific features

**Number of times user has placed an order**

In [65]:
user_n_orders = df_trunc.groupby('user_id').agg({'user_id' : 'count'}).rename(columns={'user_id' : 'count'}).reset_index()

**Mean number of days between orders**

**Median number of days between orders**

**Number of days since user's last order**

TODO: use order_number and a join to deduce this.

In [121]:
users_in_prior_only = list(set(df_trunc_prior.user_id) - set(df_trunc_train.user_id))
df_trunc_prior.loc[df_trunc_prior.user_id.isin(users_in_prior_only)] \
              .groupby('user_id')

In [132]:
df_trunc_prior.groupby('user_id').agg({'order_id': 'nunique'})

,order_id
user_id,
2,1
6,1
7,3
9,1
11,1
...,...
206203,2
206204,2
206206,4


In [133]:
df_trunc_prior.loc[df_trunc_prior.user_id == 7]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
24326535,2565571,29871,10,0,Organic Roast Beef,96,20,lunch meat,deli,7,prior,1,3,9,NaN
5361679,565870,37602,4,1,Mexican Coffee,26,7,coffee,beverages,7,prior,11,0,15,6.0
22772770,2402008,45537,13,0,Classic Round Crackers,78,19,crackers,snacks,7,prior,2,1,19,30.0


In [78]:
df_trunc.loc[df.eval_set == 'train'] \
        .groupby(['user_id', 'order_id']) \
        .agg({'days_since_prior_order':'max'}) \
        .reset_index()  # can use max or min here, it'll return the same number
# user_days_since_last_order_train = df_trunc.loc[df.eval_set == 'train'].groupby(['user_id', 'order_id']).count()
# user_days_since_last_order_prior = 

,user_id,order_id,days_since_prior_order
0,1,1187899,14.0
1,2,1492625,30.0
2,5,2196797,6.0
3,7,525192,6.0
4,8,880375,10.0
...,...,...,...
95,144,242208,12.0
96,145,3395341,30.0
97,146,673776,25.0
98,147,318441,30.0


In [ ]:
user_days_since_last_order = 

### Product-specific features

### User & product - specific features

**Number of times user has ordered this item**

In [59]:
user_item_n_orders = df_trunc.groupby(['user_id', 'order_id']).agg({'order_id':'count'}).rename(columns={'order_id' : 'count'}).reset_index()

In [60]:
user_item_n_orders.head()

,user_id,order_id,count
0,1,431534,8
1,1,473747,5
2,1,550135,5
3,1,1187899,11
4,1,2254736,5
